In [1]:
_RIOT_API_KEY='RGAPI-f133c0d1-25f4-4ab2-ad9e-7d96aa3b73f5'
from riotwatcher import LolWatcher
import helper_functions as help
import role_classification as rc
import constants
import pandas as pd
from datetime import datetime
import mysql.connector
from sqlalchemy import create_engine
import concurrent.futures

lol_watcher = LolWatcher(_RIOT_API_KEY)

1. Implement retry for when the URL gets too many requests for 1 match. probably as simple as a wrapper function around the slurps in slurp_data that retries if the clienter error exception is returned
e.g. Exception 429 Client Error: Too Many Requests for url: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_4708472978/timeline thrown. Cancelling futures.

2. Handle this exception: Exception 404 Client Error: Not Found for url: https://americas.api.riotgames.com/lol/match/v5/matches/NA1_3891774034 thrown. Cancelling futures.

3. Handling processing by puuid instead of summonerName? 
    GoofyGriefer	cT70cPTfqJ_gLJrZRqSTBO0lO3ZfM25tP4MRfvVsPw5CF5EHcdPILgojfnvBMudR9vJmnNAtFxiBvg
    CerebralPlayer	cT70cPTfqJ_gLJrZRqSTBO0lO3ZfM25tP4MRfvVsPw5CF5EHcdPILgojfnvBMudR9vJmnNAtFxiBvg
    PepegaMindset	cT70cPTfqJ_gLJrZRqSTBO0lO3ZfM25tP4MRfvVsPw5CF5EHcdPILgojfnvBMudR9vJmnNAtFxiBvg

In [2]:
engine = help.create_mysql_engine()

In [3]:
# ~25 seconds for 50 matches. More if we've hit our API Limit.
# 8m 16s for 800 records

# help.collect_riot_api_data('SouporSecret', lol_watcher, 'lol_match_timeline',
#                             engine, match_upload_limit=800, num_worker_threads=2)

In [4]:
# 23.7 seconds for 50 matches, 1 thread
# 7.4 seconds for 20 matches, 2 threads
# 5 seconds for 20 matches, 4 threads
# 4.7 seconds for 20 matches, 8 threads
# 5.8 seconds for 50 matches, 8 threads

# help.collect_riot_api_data('SouporSecret', lol_watcher, 'lol_match_details',
#                             engine, match_upload_limit=1200, num_worker_threads=2)


In [5]:
list_of_friends = ['radaemon1','koreazian','SouporSecret','SpicedCider','AerialBlast','IrishGuy',
'GoofyGriefer','LIAAAAAAAAAAAAAA','Kroocs','JellyBlaster','MegaJharo','Teroboal','Boss 0f This Gym']
for friend in list_of_friends:
    print(f"Fetching data for {friend}!")
    help.collect_riot_api_data(friend, lol_watcher, 'lol_match_timeline',
                            engine, match_upload_limit=1200, num_worker_threads=4)
    help.collect_riot_api_data(friend, lol_watcher, 'lol_match_details',
                            engine, match_upload_limit=1200, num_worker_threads=4)
rc.determine_roles(engine)

Fetching data for radaemon1!
0 matches to slurp!
0 matches to slurp!
1
Not enough roles to classify
Fetching data for koreazian!
0 matches to slurp!
0 matches to slurp!
1
Not enough roles to classify
Fetching data for SouporSecret!
1 matches to slurp!
Empty Game. Returning None for Match Timeline.
Match id NA1_4393904069 returns None.
1 matches to slurp!
Empty Game. Returning None for Match Details.
Match id NA1_4393904069 returns None.
1
Not enough roles to classify
Fetching data for SpicedCider!
1 matches to slurp!
Empty Game. Returning None for Match Timeline.
Match id NA1_4393904069 returns None.
2 matches to slurp!
Empty Game. Returning None for Match Details.
Match id NA1_4393904069 returns None.
Empty Game. Returning None for Match Details.
Match id NA1_4235418390 returns None.
1
Not enough roles to classify
Fetching data for AerialBlast!
3 matches to slurp!
Empty Game. Returning None for Match Timeline.
Match id NA1_4393904069 returns None.
Exception 404 Client Error: Not Found